In [1]:
"""
Trend Analysis and Visualization Script

This script analyzes trends in normalized metrics (Staging_z, PlotProgression_z, CognitiveTension_z) 
using various segmentation methods. It performs statistical analyses and creates combined visualizations 
for better insight into the data.

Key Features:
- ANOVA and partial eta squared analysis for Relative Segment trends.
- Fits linear and quadratic models to assess relationships between metrics and Relative Segment.
- 10x10 cross-validation for evaluating the robustness of the models.
- Generates combined plots of quadratic fits across all segmentation methods for each metric.

Folder Structure:
- Results are saved in CSV format with ANOVA results, R-squared values, and cross-validation scores.
- Combined visualizations for each metric are saved as PNG files.

Dependencies:
- Python libraries: pandas, numpy, statsmodels, seaborn, sklearn, matplotlib.

Usage:
- Ensure datasets are in CSV format and specified in the datasets list.
- Modify the metrics list to analyze other metrics.
- Run the script to generate statistical summaries and combined visualizations.

Author: Maaike de Jongh
Date: 2025-01-06
"""

import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols, glm
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Mapping for clean metric titles
metric_titles = {
    'Staging_z': 'Staging',
    'PlotProgression_z': 'Plot Progression',
    'CognitiveTension_z': 'Cognitive Tension'
}

# Result directory
execution_date = datetime.now().strftime('%Y-%m-%d')
result_directory = f"Results Trend Analysis {execution_date}"
os.makedirs(result_directory, exist_ok=True)

# Function to calculate ANOVA and partial eta squared
def calculate_anova_partial_eta(df, metric):
    """
    Calculate ANOVA and partial eta squared for a given metric.
    """
    model = ols(f"{metric} ~ Relative_Segment", data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)

    ss_total = anova_table['sum_sq'].sum()
    anova_table['partial_eta_sq'] = anova_table['sum_sq'] / ss_total

    partial_eta_squared = anova_table.loc['Relative_Segment', 'partial_eta_sq']
    return anova_table, partial_eta_squared

# Function to calculate linear and quadratic GLMs
def calculate_glm(df, metric):
    """
    Calculate linear and quadratic GLM for a given metric.
    """
    X_linear = sm.add_constant(df['Relative_Segment'])
    model_linear = sm.OLS(df[metric], X_linear).fit()

    df['Relative_Segment_Squared'] = df['Relative_Segment'] ** 2
    X_quadratic = sm.add_constant(df[['Relative_Segment', 'Relative_Segment_Squared']])
    model_quadratic = sm.OLS(df[metric], X_quadratic).fit()

    return model_linear, model_quadratic

# Function for 10x10 cross-validation
def cross_validate(df, metric, degree=1):
    """
    Perform 10x10 cross-validation for a polynomial fit model (linear or quadratic).
    """
    X = df[['Relative_Segment']].values
    if degree == 2:
        poly = PolynomialFeatures(degree=2)
        X = poly.fit_transform(X)
    y = df[metric].values

    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = []

    for _ in range(10):
        fold_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            model = LinearRegression()
            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)
            fold_scores.append(score)
        cv_scores.append(np.mean(fold_scores))

    return np.mean(cv_scores), np.std(cv_scores)

# Function to fit a quadratic GLM
def fit_quadratic_glm(df, y_var, x_var):
    glm_quadratic = glm(formula=f'{y_var} ~ np.power({x_var}, 1) + np.power({x_var}, 2)', 
                        data=df, 
                        family=sm.families.Gaussian()).fit()
    df[f'Predicted_Quadratic_{y_var}'] = glm_quadratic.fittedvalues
    return df

# Function to generate combined graphs for each metric
def plot_combined_graphs(df_list, metric, output_filename):
    """
    Generate a combined graph for a metric, including all segmentation methods.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    for df, label in df_list:
        sns.lineplot(x='Relative_Segment', 
                     y=f'Predicted_Quadratic_{metric}', 
                     data=df, 
                     label=label)

    ax.set_title(f'Quadratic Fits for {metric_titles.get(metric, metric)}', fontsize=16, color="#0209ef")
    ax.set_xlabel('Relative Segment', fontsize=14, color="#0209ef")
    ax.set_ylabel(metric_titles.get(metric, metric), fontsize=14, color="#0209ef")

    # Customize ticks, spines, and legend
    ax.tick_params(axis='x', colors="#0209ef")
    ax.tick_params(axis='y', colors="#0209ef")
    for spine in ax.spines.values():
        spine.set_color("#0209ef")

    legend = ax.legend()
    for text in legend.get_texts():
        text.set_color("#0209ef")
    
    plt.grid(True)

    # Save as PNG file
    plt.savefig(output_filename, dpi=300)
    plt.close()

# Prepare datasets and segmentation
def normalize_relative_segment(df):
    df['Relative_Segment'] = df.groupby('Filename').cumcount() / (df.groupby('Filename')['Segment'].transform('count') - 1)
    return df

datasets = [
    {'filename': '5_segments.csv', 'segment_label': '5 segments'},
    {'filename': '10_segments.csv', 'segment_label': '10 segments'},
    {'filename': '500_segments.csv', 'segment_label': '500 words'},
    {'filename': '1000_segments.csv', 'segment_label': '1000 words'}
]

metrics = ['Staging_z', 'PlotProgression_z', 'CognitiveTension_z']
results = []
df_list_for_plots = []

for dataset in datasets:
    filename = dataset['filename']
    segment_label = dataset['segment_label']
    df = pd.read_csv(filename)
    df = normalize_relative_segment(df)

    df_list_for_plots.append((df, segment_label))

    for metric in metrics:
        df = fit_quadratic_glm(df, y_var=metric, x_var='Relative_Segment')
        result = {'Segmentation': segment_label, 'Metric': metric_titles.get(metric, metric)}

        if segment_label in ['5 segments', '10 segments']:
            anova_table, partial_eta_squared = calculate_anova_partial_eta(df, metric)
            result['ANOVA p-value'] = anova_table.loc['Relative_Segment', 'PR(>F)']
            result['Partial Eta Squared'] = partial_eta_squared

        model_linear, model_quadratic = calculate_glm(df, metric)
        result['Linear GLM R2'] = model_linear.rsquared
        result['Quadratic GLM R2'] = model_quadratic.rsquared

        linear_cv_mean, linear_cv_std = cross_validate(df, metric, degree=1)
        quadratic_cv_mean, quadratic_cv_std = cross_validate(df, metric, degree=2)

        result['Linear CV Mean'] = linear_cv_mean
        result['Linear CV SD'] = linear_cv_std
        result['Quadratic CV Mean'] = quadratic_cv_mean
        result['Quadratic CV SD'] = quadratic_cv_std

        results.append(result)

# Save results as CSV
results_df = pd.DataFrame(results)
results_csv_path = os.path.join(result_directory, 'summary_results.csv')
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved in {results_csv_path}")

# Generate and save graphs
for metric in metrics:
    output_filename = os.path.join(result_directory, f'combined_graph_{metric_titles.get(metric, metric)}.png')
    plot_combined_graphs(df_list_for_plots, metric, output_filename)
    print(f"Graph saved as {output_filename}")

Results saved in Results Trend Analysis 2025-01-08/summary_results.csv
Graph saved as Results Trend Analysis 2025-01-08/combined_graph_Staging.png
Graph saved as Results Trend Analysis 2025-01-08/combined_graph_Plot Progression.png
Graph saved as Results Trend Analysis 2025-01-08/combined_graph_Cognitive Tension.png
